# Oxford international college

## Introduction to coding 

### Author

[Oliver Sheridan-Methven](mailto:oliver.sheridan-methven@maths.ox.ac.uk).

#### Date

September 2017.

## Lesson 4

> You'll notice that for this lesson there are `.py` files, 
which are regular source code files. This is dual purpose. Firstly,
source code is easier to write, test, maintain, use with 
version control systems (such as git), and is more portable.
Secondly, real code is not written for demonstrative purposes, but 
is usually designed to 'work under the hood'. Seeing source code
will better prepare you for writing real world code, and setting up 
others people's code for you to use. Nonetheless, some of this 
will be transferred into notebook format for use in class. 

### Description

In this lesson we will introduce:

 * Functions:
    - Variable and keyword arguments.
    - Partial functions. 
    - Vectorised functions.
 * Wrappers and decorators.
    - Timing.
    - Memory profiling.
    - Caching.
 * Iterators and generators. 
    - `yield`.


## Functions

### Function arguments

Often we may may have a function where we would like to extend the same functionality, but with more arguements. To see this consider the following example


In [2]:
def sum_2_numbers(x, y):
    """Sums two numbers."""
    return x + y

sum_2_numbers(1.0, 3.0)

4.0

Clearly, a more useful function would be able to take lots of arguments and add them together. 

In [3]:
def sum_n_numbers(*args):
    """Sums many numbers"""
    s = 0
    for x in args:  # "args" will be a list.
        s += x
    return s


sum_n_numbers(1, 2, 3, 4, 5)

15

What we have used here is the `*args` arguement into the function signature. This stands for variable arguements. These are optional arguments which can be added.

So what does the `*` represent? In Python, when the `*` is used it "unpacks" a list (or similar). Consider:

In [4]:
sum_n_numbers(*range(1000))

499500

Here

```
sum_n_numbers(*range(1000))
```

This is equivalent to 

```
sum_n_numbers(0, 1, 2, 3, ..., 998, 999)
```

#### Default and keyword arguements.

Usually when we write a function, we may have some parameters on which the function depends. Similarly, we might like to suggest default values to the user. Consider the following example, where we print a name to the console, and we suggest some default value. This is acheived by decalring what the default value should equal in the function's definition.

In [5]:
def print_my_name(name='Oliver'):
    """Prints a name."""
    print name

This can be called in several different ways. 

In [6]:
print_my_name()  # Use the default value.
print_my_name('David')  # Specify the first argument of the function.
print_my_name(name='Raphael')  # Provide the value of the argument by naming it. (Order no longer important).

Oliver
David
Raphael


Now recall that we could specify an arbitrary list of unnamed arguments using `*args`, and similarly we can also unpack a list of several named arguements, and this is achieved using `**kwargs`, (keyword arguments). Now as these come in a `name:value` pair, a natural container is a dictionary.

> Only the `*` or `**` are important in the function signature, so although the convention is to call these `args` and `kwargs`, is there is a more descriptive variable name, then maybe use it. However, usually you should stick to `args` and `kwargs`, and this is what everyone usually uses!

In [8]:
def print_names_and_ages(**names):
    """Prints names and ages."""
    for k, v in names.iteritems():
        print('Name: {:<10}\tAge: {}\n'.format(k, v))

In [10]:
print_names_and_ages(Daniel=24, 
                     Federico=26, 
                     Joseph=33, 
                     **{'Oliver': 24, 'Susan': 52})


Name: Joseph    	Age: 33

Name: Daniel    	Age: 24

Name: Oliver    	Age: 24

Name: Susan     	Age: 52

Name: Federico  	Age: 26



Now, we can combine mandatory arguements, and optional arguements together in the same function signature. This takes the form:

```
def function_name(mandatory_arguments, 
                  default_value_arguments,
                  *args,
                  **kwargs)
    Do something.
```

In [12]:
def example_with_all_arguments(x, y=1, *args, **kwargs):
    """
    An example including:
    :param x: Float, mandatory argument.
    :param y: Float, mandatory argument (default y=1).
    :param args: Floats, extra optional argument.
    :param kwargs: Dict, extra optional argument.
    :return: Something.
    """
    z = x + y + sum(args)
    for k, v in kwargs.iteritems(): print k, v
    return z


example_with_all_arguments(2, 3, 4, 5, instructor='Oliver', students='OIC')

students OIC
instructor Oliver


14

> In the above example, the keyword arguements are declared at the end of the function call. Have a think about why this is the case, and why this must be the case. 

### Partial functions

Consider a function which has some default value:

In [20]:
from collections import Iterable
import numpy as np

def norm(x, metric='l2'):
    """
    Computes the different l-norms.
    :param x: Iterable, vector.
    :param metric: Str or Float, normalisation metric.
    :return: Float, vector norm.
    """
    assert isinstance(x, Iterable), 'Please enter an iterable object to compute the norm over.'
    defined_norms = ('l0', 'l1', 'l2', 'inf')  # And positive floats.
    assert metric in defined_norms or metric > 0, 'The metric {} must be either {} or a positive float.'.format(metric, defined_norms)
    r = 0.0
    for i in x:
        if metric == 'l0':
            r += i != 0
        elif metric == 'l1':
            r += abs(i)
        elif metric == 'l2':
            r += i ** 2.0
        elif metric == 'inf':
            r = max(i, r)
        else:
            r += i ** metric

    if metric == 'l0':
        pass
    elif metric == 'l1':
        pass
    elif metric == 'l2':
        r = r ** 0.5
    elif metric == 'inf':
        pass
    else:
        r = pow(r, 1.0 / metric)

    return r

x = np.array([1.0, 2.0, 3.0, 4.0])

print norm(x)  # Use the default value
print norm(x, metric='l2')  # Explicitly set the metric.


5.47722557505
5.47722557505


Supposing we wanted to declare a variant of this function which has some different default value for one or more of its arguements. This can be acieved by creating a partial function. The `partial` function is from the `functools` module. 

In [23]:
from functools import partial

l_1_norm = partial(norm, metric='l1')  # Redefine the default value.
l_2_norm = partial(norm, metric='l2')
l_inf_norm = partial(norm, metric='inf')

print x
print l_1_norm(x)
print l_2_norm(x)
print l_inf_norm(x)

[ 1.  2.  3.  4.]
10.0
5.47722557505
4.0


### Vectorised functions

Vectorised functions are useful when we have a function which was originally designed to only work on one item, but we would like ito to be able to work on many items and return a list of results. Consider the following example of a one item function:

In [24]:
def is_a_triangle_number(x):
    """
    Computes whether a number is a triangle number.
    :param x: Int.
    :return: Bool.
    """
    assert isinstance(x, int) and x > 0, 'Please input an integer.'
    i = 1
    while x > 0:
        x -= i
        i += 1
    return x == 0


for i in range(1, 11): print i, is_a_triangle_number(i)

1 True
2 False
3 True
4 False
5 False
6 True
7 False
8 False
9 False
10 True


We use the `np.vectorize` function wrapper to achieve a vectorised function:


In [26]:
vectorised_function = np.vectorize(is_a_triangle_number)
vectorised_function(range(1, 11))

array([ True, False,  True, False, False,  True, False, False, False,  True], dtype=bool)

> Try to design functions so they are already vectorised wherever possible, as this is not a perfect remidy. E.g. ` assert type(x) == int, 'Please input an integer.'` won't work with `np.vectorise`. This also highlights that for type checking, is it better to use `isinstance` rather than `type(x) == ...`!

Alternatives to `np.vectorize` are python's inbuilt `map`, or list comprehension. The performance differences between these are usually case dependent, so instead concern yourself with readability!

> `map` is part of the functional programming group of built in functions. Similar items in this group include `filter` and `reduce`, which can sometime be shorter than writing out list comprehensions. 

## Wrappers and decorators

Wrappers and decorators are designed to allow a programmer to easily extend the functionality of a function (or class) easy, without interfering with the core functionality. We have already encounted an example of a function decorator with `np.vectorize`. The basic syntax for a decorator is:

```
decorator(function) -> new function
```
 

In [54]:
import time

def timeit(method):
    def timed(*args, **kwargs):
        t_start = time.time()
        result = method(*args, **kwargs)
        t_end = time.time()
        print '%r took %2.5f s' % \
              (method.__name__, t_end - t_start)
        return result
    return timed

x = np.random.normal(0, 1, 10 ** 6)

max_decorated = timeit(max)

max_decorated(x)


'max' took 0.05598 s


4.8991306521819107

A wrapper is a more general form of a decorator, in which the decorating function might inself have some functional dependence. A wrapper takes the following form:

In [55]:
def wrap(pre, post):
    def decorate(func):
        def call(*args, **kwargs):
            pre(func, *args, **kwargs)
            result = func(*args, **kwargs)
            post(func, *args, **kwargs)
            return result
        return call
    return decorate

This can be called with optional functionality, changing how the wrapper works. 

The key benefit of wrappers and decorators is that they can be called easily, and support the Python `@` decoration syntax:

In [58]:
def trace_in(func, *args, **kwargs):
   print "Entering function",  func.__name__

def trace_out(func, *args, **kwargs):
   print "Leaving function", func.__name__

@wrap(trace_in, trace_out)
def sum_two_numbers(x, y):
   return x + y

sum_two_numbers(1, 2)

Entering function sum_two_numbers
Leaving function sum_two_numbers


3

## Iterators and generators

We have already encountered iterators, e.g. lists.


In [28]:
y = [1, 2, 3, 4, 5]

for i in y:
    print i

1
2
3
4
5


Here `y` is an iterator, and is computed and stored in memory for the entire duriation of the `for` loop.

Suppose we wanted to iterate through something more interesting, but generate the iterates as we need them, and forget them when we no longer need them:

In [33]:
s = [j ** 2 for j in range(5)]  # Iterator
g = (j ** 2 for j in range(5))  # Generator

These behave much the same:

In [34]:
for x in s:
    print x

for x in g:  # We compute the next element in 'g' as we go along.
    print x

0
1
4
9
16
0
1
4
9
16


> Using generators becomes extremely useful if the process is time or memory expensive.

Suppose we wanted something to generate the prime numbers, and we introduced the following prime number generator

> NB - There is an entire field of mathematics and computing devoted the the generation of prime numbers, and this is only to demonstrate when we might use a generator.

In [35]:
def prime_numbers():
    """
    A prime number generator.
    :return: Int, prime number.
    """
    x = 2
    primes = [x]
    yield x  # We know at least the first prime number.
    while True:
        if x in primes:
            x += 1
        else:
            is_prime = True  # We assume the number might be prime
            for p in primes:
                if x % p == 0:
                    is_prime = False  # The number clearly is not prime.
                    x += 1
                    break  # We do not need to check against any other prime numbers.
            if is_prime:
                primes.append(x)
                yield x

The above looks a lot like a function, but we notice that we have `yield` where we might have otherwise expected `return`. This signifies to Python that this is not a function but a generator. The significant of this is that when the generator is called again, the generator will simply carry on where it left of (all the function variables are remembered, and aren't deleted by the garbage collector!). 

> With these types of generators which might never end, be sure that you have code which will terminate eventually. 

In [37]:
i = 1
for p in prime_numbers():
    print p
    i += 1
    if i > 10:
        break

2
3
5
7
11
13
17
19
23
29


If we call the same generator again, it will actually construct a new generator, and so will start from the beginiining again:

In [38]:
i = 1
for p in prime_numbers():
    print p
    i += 1
    if i > 5:
        break
        
# And we re-use the generator from the beginning.

i = 1
for p in prime_numbers():
    print p
    i += 1
    if i > 5:
        break    

2
3
5
7
11
2
3
5
7
11


If instead we wanted to continue using the generator, then we must create a single instance, and then refer to this instance:



In [41]:
g = prime_numbers()

i = 1
for p in g:
    print p
    i += 1
    if i > 5:
        break
        
# We continue with the same generator where we left off. 

i = 1
for p in g:
    print p
    i += 1
    if i > 5:
        break

2
3
5
7
11
13
17
19
23
29


If a function contains the word `yield` *anywhere* in its definition, then it is a generator, not a function. This is even the case if the `yield` is inaccessible!

If we mix `yield` and `return` in the definition, then the return signifies that the generator should stop, which is very useful! If the interpretor reaches the end of the generator's defining code, then it will return. Consider the following extension of the prime number generator:

In [42]:

def n_prime_numbers(n=None, m=None):
    """
    A prime number generator.
    :param n: Int, number of prime numbers to return.
    :param m: Int, inclusive upper bound.
    :return: Int, prime number.

    Note:

        If both "n" and "m" are specified, then whichever is achieved first will cause the termination.
        Hence, if "n" is very large and "m" is small, then "m" will take precedence.
    """
    assert n is None or (isinstance(n, int) and n >= 0), 'Please enter a positive integer for the number of prime numbers.'
    assert m is None or m >= 0, 'Please enter a positive upper bounding number.'
    if m is not None:
        m = int(m)
    if n == 0:
        return
    x = 2  # We know the first prime number.
    if m is not None and m < x:
        return
    primes = [x]
    n_primes = 1  # We could recompute the length every time, but using a counter is easier.
    yield x
    while True:
        if x in primes:
            x += 1
        else:
            is_prime = True  # We assume the number might be prime
            for p in primes:
                if x % p == 0:
                    is_prime = False  # The number clearly is not prime.
                    x += 1
                    break  # We do not need to check against any other prime numbers.
            if is_prime:
                primes.append(x)
                n_primes += 1
                yield x
        if m is not None and m < x:
            return
        elif n is not None and n_primes >= n:
            return

for p in n_prime_numbers(n=10):
    print p

2
3
5
7
11
13
17
19
23
29
